<a href="https://colab.research.google.com/github/onmax/bike-forecasting/blob/main/src/preprocessing/dataset-creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Getting the dataset
!git clone "https://github.com/onmax/bike-forecasting"
%cd bike-forecasting/src/preprocessing

Cloning into 'bike-forecasting'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.
Checking out files: 100% (35/35), done.
/content/bike-forecasting


In [1]:
import pandas as pd
import os
import pickle


# files got from https://divvy-tripdata.s3.amazonaws.com/index.html. Data folder should look like this
'''
├── data
│   ├── files
│   │   ├── 2014
│   │   │   └── ...
│   │   ├── 2015
│   │   │   └── ...
│   │   ├── 2016
│   │   │   └── ...
│   │   ├── 2017
│   │   │   └── ...
│   │   ├── 2018
│   │   │   └── ...
│   │   └── 2019
│   │       └── ...
│   └── raw
│       ├── Divvy_Stations_Trips_2014_Q1Q2.zip
│       ├── Divvy_Stations_Trips_2014_Q3Q4.zip
│       ├── Divvy_Trips_2015-Q1Q2.zip
│       ├── Divvy_Trips_2015_Q3Q4.zip
│       ├── Divvy_Trips_2016_Q1Q2.zip
│       ├── Divvy_Trips_2016_Q3Q4.zip
│       ├── Divvy_Trips_2017_Q1Q2.zip
│       ├── Divvy_Trips_2017_Q3Q4.zip
│       ├── Divvy_Trips_2018_Q1.zip
│       ├── Divvy_Trips_2018_Q2.zip
│       ├── Divvy_Trips_2018_Q3.zip
│       ├── Divvy_Trips_2018_Q4.zip
│       ├── Divvy_Trips_2019_Q1.zip
│       ├── Divvy_Trips_2019_Q2.zip
│       ├── Divvy_Trips_2019_Q3.zip
│       └── Divvy_Trips_2019_Q4.zip
└── src 
    └── ...
'''

'\n├── data\n│\xa0\xa0 ├── files\n│\xa0\xa0 │\xa0\xa0 ├── 2014\n│\xa0\xa0 │\xa0\xa0 │   └── ...\n│\xa0\xa0 │\xa0\xa0 ├── 2015\n│\xa0\xa0 │\xa0\xa0 │   └── ...\n│\xa0\xa0 │\xa0\xa0 ├── 2016\n│\xa0\xa0 │\xa0\xa0 │   └── ...\n│\xa0\xa0 │\xa0\xa0 ├── 2017\n│\xa0\xa0 │\xa0\xa0 │   └── ...\n│\xa0\xa0 │\xa0\xa0 ├── 2018\n│\xa0\xa0 │\xa0\xa0 │   └── ...\n│\xa0\xa0 │\xa0\xa0 └── 2019\n│\xa0\xa0 │\xa0\xa0     └── ...\n│\xa0\xa0 └── raw\n│\xa0\xa0     ├── Divvy_Stations_Trips_2014_Q1Q2.zip\n│\xa0\xa0     ├── Divvy_Stations_Trips_2014_Q3Q4.zip\n│\xa0\xa0     ├── Divvy_Trips_2015-Q1Q2.zip\n│\xa0\xa0     ├── Divvy_Trips_2015_Q3Q4.zip\n│\xa0\xa0     ├── Divvy_Trips_2016_Q1Q2.zip\n│\xa0\xa0     ├── Divvy_Trips_2016_Q3Q4.zip\n│\xa0\xa0     ├── Divvy_Trips_2017_Q1Q2.zip\n│\xa0\xa0     ├── Divvy_Trips_2017_Q3Q4.zip\n│\xa0\xa0     ├── Divvy_Trips_2018_Q1.zip\n│\xa0\xa0     ├── Divvy_Trips_2018_Q2.zip\n│\xa0\xa0     ├── Divvy_Trips_2018_Q3.zip\n│\xa0\xa0     ├── Divvy_Trips_2018_Q4.zip\n│\xa0\xa0     ├── D

In [42]:
def path():
    current_path = os.getcwd()
    return f"{current_path}/../../data/files"

def data_path(year):
    return f"{path()}/{year}"

def get_csvs(years):
    current_path = path()
    paths = []
    for year in years:
        pickle_path = f"{current_path}/{year}/trips-{year}.csv"
        paths.append(pickle_path)
    return paths

def merge_csv(inputs, output, year, cols):
    df = pd.DataFrame()
    for input in inputs:
        print(f"Reading {input}")
        df_temp = pd.read_csv(input)
        df_temp[cols[0]] = pd.to_datetime(df_temp[cols[0]], format='%Y-%m-%d %H:%M:%S', infer_datetime_format=True)
        df = pd.concat([df, df_temp], join='outer')
    output_file = f"{output}/trips-{year}.csv"
    print(f"Writing {output_file}")
    df[cols].to_csv(output_file,index=False)

def merge_years(inputs):
    df = pd.DataFrame()
    for input in inputs:
        print(f"Reading {input}")
        df_temp = pd.read_csv(input)
        df = pd.concat([df, df_temp], join='outer')
    df['start_time'] = df['start_time'].combine_first(df['starttime'])
    df = df.drop(columns=["starttime"])
    return df

def save_with_split(df, dest_folder, write_size):
    # Make a destination folder if it doesn't exist yet
    if not os.path.exists(dest_folder):
        os.mkdir(dest_folder)
    else:
        # Otherwise clean out all files in the destination folder
        for file in os.listdir(dest_folder):
            os.remove(os.path.join(dest_folder, file))
    partnum = 0

    df.to_csv(f"{path()}/group.csv")
    
    with open(f"{path()}/group.csv", 'rb') as input:
        while True:
            chunk = input.read(write_size)
            if not chunk:
                # End the loop if we have hit EOF
                break
            partnum += 1

            print(f"Writing {dest_folder}/chicago-divvy-trips-part-{partnum}")
            # Create a new file name
            with open(f"{dest_folder}/chicago-divvy-trips-part-{partnum}", 'wb') as fd:
                fd.write(chunk)

def join(output_file, parts):
  with open(output_file, 'wb') as output:
    for part in parts:
        with open(part, 'rb') as input_file:
          print(f"Putting together {part}")
          output.write(input_file.read())

In [24]:
year = "2014"
csv_folder = data_path(year)
csv_names = ["Divvy_Trips_2014_Q1Q2.csv", "Divvy_Trips_2014-Q3-07.csv", "Divvy_Trips_2014-Q3-0809.csv", "Divvy_Trips_2014-Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["starttime", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014_Q1Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014-Q3-07.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014-Q3-0809.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/Divvy_Trips_2014-Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/trips-2014.csv


In [32]:
year = "2015"
csv_folder = data_path(year)
csv_names = ["Divvy_Trips_2015-Q1.csv", "Divvy_Trips_2015-Q2.csv", "Divvy_Trips_2015_07.csv", "Divvy_Trips_2015_08.csv", "Divvy_Trips_2015_09.csv", "Divvy_Trips_2015_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["starttime", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015-Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015-Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_07.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_08.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_09.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/Divvy_Trips_2015_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/trips-2015.csv


In [25]:
year = "2016"
csv_folder = data_path(year)
csv_names = ["Divvy_Trips_2016_Q1.csv", "Divvy_Trips_2016_04.csv", "Divvy_Trips_2016_05.csv", "Divvy_Trips_2016_06.csv","Divvy_Trips_2016_Q3.csv", "Divvy_Trips_2016_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["starttime", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_04.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_05.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_06.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/Divvy_Trips_2016_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/trips-2016.csv


In [26]:
year = "2017"
csv_folder = data_path(year)
csv_names = ["Divvy_Trips_2017_Q1.csv", "Divvy_Trips_2017_Q2.csv", "Divvy_Trips_2017_Q3.csv", "Divvy_Trips_2017_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["start_time", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/Divvy_Trips_2017_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/trips-2017.csv


In [27]:
year = "2018"
csv_folder = data_path(year)
csv_names = ["Divvy_Trips_2018_Q1.csv", "Divvy_Trips_2018_Q2.csv", "Divvy_Trips_2018_Q3.csv", "Divvy_Trips_2018_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["start_time", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/Divvy_Trips_2018_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/trips-2018.csv


In [28]:
year = "2019"
csv_folder = data_path(year)
csv_names = ["Divvy_Trips_2019_Q1.csv", "Divvy_Trips_2019_Q2.csv", "Divvy_Trips_2019_Q3.csv", "Divvy_Trips_2019_Q4.csv"]
csv_paths = [f"{csv_folder}/{f}" for f in csv_names]
merge_csv(csv_paths, csv_folder, year, ["start_time", "from_station_id", "from_station_name"])

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q1.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q2.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q3.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/Divvy_Trips_2019_Q4.csv
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/trips-2019.csv


In [33]:
# Merge all pickles
years = list(range(2014, 2020))
csvs = get_csvs(years)
df = merge_years(csvs)
df

Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2014/trips-2014.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2015/trips-2015.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2016/trips-2016.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2017/trips-2017.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2018/trips-2018.csv
Reading /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/2019/trips-2019.csv


,from_station_id,from_station_name,start_time
0,131,Lincoln Ave & Belmont Ave,2014-06-30 23:57:00
1,282,Halsted St & Maxwell St,2014-06-30 23:56:00
2,327,Sheffield Ave & Webster Ave,2014-06-30 23:33:00
3,134,Peoria St & Jackson Blvd,2014-06-30 23:26:00
4,320,Loomis St & Lexington St,2014-06-30 23:16:00
...,...,...,...
3817999,112,Green St & Randolph St,2019-12-31 23:56:13
3818000,90,Millennium Park,2019-12-31 23:56:34
3818001,623,Michigan Ave & 8th St,2019-12-31 23:57:05
3818002,623,Michigan Ave & 8th St,2019-12-31 23:57:11


In [35]:
# Group trips by hour

df["start_time"] = pd.to_datetime(df["start_time"], format='%Y-%m-%d %H:%M:%S')

INTERVAL = "1H" # It could be also 15Min
df = df.groupby('from_station_id').resample(INTERVAL, on='start_time') \
        .size() \
        .to_frame() \
        .rename(columns={0: "quantity"}) \
        .reset_index() \
        .set_index("start_time")
df


,from_station_id,quantity
start_time,,
2019-05-21 14:00:00,1,1
2015-05-08 12:00:00,2,7
2015-05-08 13:00:00,2,0
2015-05-08 14:00:00,2,1
2015-05-08 15:00:00,2,2
...,...,...
2019-12-31 13:00:00,673,0
2019-12-31 14:00:00,673,0
2019-12-31 15:00:00,673,0


In [43]:
save_with_split(df=df, write_size=49500000, dest_folder=f'{path()}/../parts')

Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-1
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-2
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-3
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-4
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-5
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-6
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-7
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-8
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago-divvy-trips-part-9
Writing /mnt/f/proyectos/bikes/src/preprocessing/../../data/files/../parts/chicago

In [39]:
parts = [f"{path()}/../parts/chicago-divvy-trips-part-{i}" for i in list(range(1,34))]
join(f'{path()}/trips.csv', parts)

Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-2
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-3
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-4
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-5
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-6
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-7
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-8
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-9
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-10
Putting together /content/bike-forecasting/src/preprocessing/../../data/files/parts/trips-part-11
Putting together /content/bi